Section 3.2: Database implementation

Rajat Nepal

In [25]:
from google.colab import drive
drive.mount('/content/drive')

! apt-get update
! apt-get install mysql-server 

!mysql --version   # Making sure everything has been installed correctly by checking the version
!service mysql start   # Starting our server

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/ubunt

In [26]:
%%bash
#making sure I am in right folder and data is there
cd ~
cd /content/drive/MyDrive/project
ls

article-ids.txt
parse_article.py
parse_article.sh
wikipedia-ml
wikipedia-ml-raw


In [27]:
!mysql

Welcome to the MySQL monitor.  Commands end with ; or \g.
Your MySQL connection id is 10
Server version: 8.0.32-0ubuntu0.20.04.2 (Ubuntu)

Copyright (c) 2000, 2023, Oracle and/or its affiliates.

Oracle is a registered trademark of Oracle Corporation and/or its
affiliates. Other names may be trademarks of their respective
owners.

Type 'help;' or '\h' for help. Type '\c' to clear the current input statement.

mysql> create database projectDB;
Query OK, 1 row affected (0.01 sec)

mysql> use projectDB;
Database changed
mysql> CREATE TABLE Article (id INT AUTO_INCREMENT, year INT, month INT, day INT , articleBody TEXT, references TEXT, PRIMARY KEY (id));
ERROR 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'references TEXT, PRIMARY KEY (id))' at line 1
mysql> CREATE TABLE Article (id INT AUTO_INCREMENT, year INT, month INT, day INT , articleBody TEXT, references TEXT);
ERROR 1064 (42000

mysql commands used in the previous cell:

create database projectDB;

use projectDB;

CREATE TABLE Article (id INT AUTO_INCREMENT, year INT, month INT, day INT, articleBody TEXT, refs TEXT,  PRIMARY KEY (id));

show tables;

exit;



Below is my bash code (I could have and should have had it in a bash file and ran it as a script, but I put it here so it would be easier for you to read it and not have to open a new file). The comments show what is happening at each chunk of code

In [52]:
%%bash

cd ~
cd /content/drive/MyDrive/project/wikipedia-ml

for article_file in article_*.txt;
do 
  #here we are extracting the article dates and body
  #used the sed command: https://www.geeksforgeeks.org/sed-command-in-linux-unix-with-examples/
  year=$(echo $article_file | sed 's/article_\([^_]*\)_\([0-9]*\)_\([0-9]*\)_\([0-9]*\).txt/\2/')
  month=$(echo $article_file | sed 's/article_\([^_]*\)_\([0-9]*\)_\([0-9]*\)_\([0-9]*\).txt/\3/')
  day=$(echo $article_file | sed 's/article_\([^_]*\)_\([0-9]*\)_\([0-9]*\)_\([0-9]*\).txt/\4/')
  article_body=$(cat "$article_file")
  #here we are finding the corresponding reference file and storing its contents
  ref_file="refs_Machine learning - Wikipedia_${year}_${month}_${day}.txt"
  ref_body=$(cat "$ref_file")

  #have to do this to escape the special characters becore putting it into the sql database
  article_body=$(echo $article_body | sed "s/'/\\\'/g")
  article_body=$(echo $article_body | sed 's/"/\\"/g')
  ref_body=$(echo $ref_body | sed "s/'/\\\'/g")
  ref_body=$(echo $ref_body | sed 's/"/\\"/g')
  #(year, month, day, articleBody, refs) values ($year, $month, $day, '$article_body', '$references')
  mysql -D "projectDB" -e "insert into Article (year, month, day, articleBody, refs) values ($year, $month, $day, '$article_body', '$references')" 

done;






Just to make sure that all the data is there, I printed the top 4 rows using the sql command prompt thingy

In [53]:
!mysql

Welcome to the MySQL monitor.  Commands end with ; or \g.
Your MySQL connection id is 74
Server version: 8.0.32-0ubuntu0.20.04.2 (Ubuntu)

Copyright (c) 2000, 2023, Oracle and/or its affiliates.

Oracle is a registered trademark of Oracle Corporation and/or its
affiliates. Other names may be trademarks of their respective
owners.

Type 'help;' or '\h' for help. Type '\c' to clear the current input statement.

mysql> help

For information about MySQL products and services, visit:
   http://www.mysql.com/
For developer information, including the MySQL Reference Manual, visit:
   http://dev.mysql.com/
To buy MySQL Enterprise support, training, or other products, visit:
   https://shop.mysql.com/

List of all MySQL commands:
Note that all text commands must be first on line and end with ';'
?         (\?) Synonym for `help'.
clear     (\c) Clear the current input statement.
connect   (\r) Reconnect to the server. Optional arguments are db and host.
delimiter (\d) Set statement delimiter.
e

In [54]:
#this selects the distinct years from the data table
!mysql -D "projectDB" -e "select distinct year from Article;" 

+------+
| year |
+------+
| 2013 |
| 2014 |
| 2015 |
| 2016 |
| 2017 |
| 2018 |
| 2019 |
| 2020 |
| 2021 |
| 2022 |
| 2023 |
+------+


In [55]:
#this code gets the number of documents in each year with a simple group by query
!mysql -D "projectDB" -e "select year, count(*) as num_document from Article group by year;" 

+------+--------------+
| year | num_document |
+------+--------------+
| 2013 |            2 |
| 2014 |            2 |
| 2015 |            2 |
| 2016 |            2 |
| 2017 |            2 |
| 2018 |            2 |
| 2019 |            2 |
| 2020 |            2 |
| 2021 |            2 |
| 2022 |            2 |
| 2023 |            1 |
+------+--------------+


Here I am getting the count of the distinct years each of the terms appeared. I did this by using count(distinct year)(which is pretty self-explanatory). and using regex to find if the body has the term

In [56]:
!mysql -D "projectDB" -e "select count(distinct year) as num_years_supervised from Article where articleBody like '%supervised%'"

+----------------------+
| num_years_supervised |
+----------------------+
|                   11 |
+----------------------+


In [57]:
!mysql -D "projectDB" -e "select count(distinct year) as num_years_semi_supervised from Article where articleBody like '%semi-supervised%'"

+---------------------------+
| num_years_semi_supervised |
+---------------------------+
|                         8 |
+---------------------------+


In [58]:
!mysql -D "projectDB" -e "select count(distinct year) as num_years_unsupervised from Article where articleBody like '%unsupervised%'"

+------------------------+
| num_years_unsupervised |
+------------------------+
|                     11 |
+------------------------+


In [59]:
!mysql -D "projectDB" -e "select count(distinct year) as num_years_meta_learning from Article where articleBody like '%meta-learning%'"

+-------------------------+
| num_years_meta_learning |
+-------------------------+
|                       2 |
+-------------------------+


Here I am frinding the earliest year each term cam about. Very similar to the previous sql commands, except instead of having count(distinct year), i have the min(year)

In [60]:
!mysql -D "projectDB" -e "select min(year) as earliest_year_supervised from Article where articleBody like '%supervised%'"

+--------------------------+
| earliest_year_supervised |
+--------------------------+
|                     2013 |
+--------------------------+


In [61]:
!mysql -D "projectDB" -e "select min(year) as earliest_year_semi_supervised from Article where articleBody like '%semi-supervised%'"

+-------------------------------+
| earliest_year_semi_supervised |
+-------------------------------+
|                          2015 |
+-------------------------------+


In [62]:
!mysql -D "projectDB" -e "select min(year) as earliest_year_unsupervised from Article where articleBody like '%unsupervised%'"

+----------------------------+
| earliest_year_unsupervised |
+----------------------------+
|                       2013 |
+----------------------------+


In [63]:
!mysql -D "projectDB" -e "select min(year) as earliest_year_meta_learning from Article where articleBody like '%meta-learning%'"

+-----------------------------+
| earliest_year_meta_learning |
+-----------------------------+
|                        2022 |
+-----------------------------+


Here I am calculating the number of references per year if they were separated by a \n charachter. However, I don't think the \n charachter is present when i inserted the info into my database. This is probably because I inserted it as a text and not a blob data type. the blob data type would have kept \n but I guess the text data type did not.

I believe my logic in the code below is still sound however. I am getting the lengths of the refs attribute with and without the \n charachter, getting the difference and adding 1 (since the first reference would not have a \n at the beinning). I am then grouping and summing it (for each year)

In [64]:
!mysql -D "projectDB" -e "select year, sum(length(refs) - length(replace(refs,'\n', '')) +1) as num_references from Article group by year;"

+------+----------------+
| year | num_references |
+------+----------------+
| 2013 |              2 |
| 2014 |              2 |
| 2015 |              2 |
| 2016 |              2 |
| 2017 |              2 |
| 2018 |              2 |
| 2019 |              2 |
| 2020 |              2 |
| 2021 |              2 |
| 2022 |              2 |
| 2023 |              1 |
+------+----------------+
